---
# Demo Use Case (Part 2)

The video is now being continuously ingested into SDP.

As each video frame is ingested, the object detection model is used to detect objects such as cars, buses, and people.
The object detection model runs as a Flink job in SDP. It uses the popular YOLO object detection model.
The Flink job uses Java bindings for TensorFlow and it can be GPU-accelerated when GPUs are available on the host.
In this demo, the CPU is used for inference.

See SDP UI.

Now the data scientist wants to view the result of the object detection model in real-time.
This is shown as a sequence of video frames with labeled boxes around detected objects.

---

### Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream
from matplotlib import pyplot as plt
import os

imp.reload(video);

### Define Pravega stream parameters

In [ ]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
#stream = 'object-detector-input-video'
stream = 'object-detector-output-video'
#stream = 'generated-video'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### View the size of the video stream

In [ ]:
unindexed_stream = UnindexedStream(pravega_client, scope, stream)

In [ ]:
stream_info = unindexed_stream.get_stream_info()
stream_info

In [ ]:
stream_size_MB = (list(stream_info.tail_stream_cut.cut.values())[0] - list(stream_info.head_stream_cut.cut.values())[0]) * 1e-6
stream_size_MB

### Play video stream from the current time

In [ ]:
stream_info = unindexed_stream.get_stream_info()
from_stream_cut = stream_info.tail_stream_cut
to_stream_cut = None
figheight = 5
unindexed_stream.play_video(from_stream_cut, to_stream_cut, show_frame_interval=10, figsize=(figheight*16/9,figheight))

# Play video stream from the earliest time

In [ ]:
figheight = 3
unindexed_stream.play_video(from_stream_cut=None, to_stream_cut=None, show_frame_interval=1, figsize=(figheight*16/9,figheight))

# Play video stream from the earliest time and stop at current time

In [ ]:
stream_info = unindexed_stream.get_stream_info()
from_stream_cut = stream_info.head_stream_cut
to_stream_cut = stream_info.tail_stream_cut
figheight = 10
unindexed_stream.play_video(from_stream_cut, to_stream_cut, show_frame_interval=1, figsize=(figheight*16/9,figheight))

# Play video stream from a stream cut

In [ ]:
from_stream_cut_text = 'H4sIAAAAAAAAADOwSq1IzC3ISS3Wz0/KSk0u0U1JLQFS+UW6+aUlBaUlumWZKan5VgZAaGhoamJqYGxpYQIA+ZiLQTYAAAA='
from_stream_cut = pravega.pb.StreamCut(text=from_stream_cut_text)
to_stream_cut = None
figheight = 2
unindexed_stream.play_video(from_stream_cut, to_stream_cut, show_frame_interval=3, figsize=(figheight*16/9,figheight))